In [10]:
import geopandas as gpd
import pandas as pd
import sys
import json
import numpy as np
import rasterio as rs
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
#from rasterstats import zonal_stats

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def reproject_to_meters(input_raster, output_raster, target_crs='EPSG:3857'):
    
    with rs.open(input_raster) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rs.open(output_raster, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rs.band(src, i),
                    destination=rs.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )

In [13]:
prj_districts = gpd.read_file('../../data/epa_districts/Project_Districts.shp')
input_raster = '/Users/jessica.ertel/github/plantation_classifier/tmp/ghana/preds/mosaic/final_2024-10-29.tif'
repj_raster = '/Users/jessica.ertel/github/plantation_classifier/tmp/ghana/preds/mosaic/reprojected_2024-10-29.tif'

# reproject inputs from degrees to meters for area assessment calcs
dst_crs = 'EPSG:3857'
reproject_to_meters(input_raster, repj_raster, dst_crs)
prj_districts = prj_districts.to_crs(dst_crs)

In [28]:
def calculate_adjusted_area(repr_raster, prj_districts, error_dict):
    '''
    Requires raster that has been reprj to meter CRS, a shapefile
    for the 26 priority districts, and a dictionary containing error
    statistics for each land use class
    Calculates area assessment in ha for each land use class in each
    district and adjusts assessments based on model error
    '''
    
    zonal_stats = []
    
    with rs.open(repr_raster) as src:
        for _, district in prj_districts.iterrows():
            out_image, out_transform = mask(src, [district.geometry], crop=True)
            district_mask = out_image[0] 
            unique, counts = np.unique(district_mask, return_counts=True)
            land_use_stats = dict(zip(unique, counts))

            # Calculate pixel size to convert to hectares
            pixel_width = src.transform[0]  # X resolution
            pixel_height = -src.transform[4]  # Y resolution
            px_size = pixel_width * pixel_height
            land_use_stats = {k: v * (px_size / 10000) for k, v in land_use_stats.items()}
            
            # Add district name
            land_use_stats['district'] = district.ADM2_EN
            zonal_stats.append(land_use_stats)

    df = pd.DataFrame(zonal_stats)
    df = df.round(2).rename(columns={
        0: "No vegetation",
        1: "Monoculture",
        2: "Agroforestry",
        3: "Natural",
        255: "No data"
    })
    
    # Adjust for error
    for land_use_class, stats in error_dict.items():
        if isinstance(stats, dict) and 'adj' in stats:
            if land_use_class in df.columns:
                df[land_use_class] = (df[land_use_class] * stats['adj']).round()

    df.to_csv('../../data/adj_area_assessment.csv', index=False)
    return df

In [39]:
with open("../../data/validation/ci_error_adjustment.json", "r") as f:
    error = json.load(f)

error

{'overall_accuracy': [0.6592439785905442, 0.02855709188224792],
 'No vegetation': {'recall': [0.8985618566452718, 0.02832493580755846],
  'precision': [0.7037842782867856, 0.03959309869211636],
  'adj': 1.276757501364809},
 'Monoculture': {'recall': [0.7998529411764705, 0.2001470588235294],
  'precision': [0.25784595447374353, 0.11993182330403425],
  'adj': 3.102057361376673},
 'Natural': {'recall': [0.4593699882991718, 0.060963613797179794],
  'precision': [0.5824315515908155, 0.06732098648025042],
  'adj': 0.7887106854779392},
 'Agroforestry': {'recall': [0.5345372981917225, 0.050009402659326324],
  'precision': [0.6968495208999447, 0.05234712456568846],
  'adj': 0.7670770835881404}}

In [29]:
area_assess = calculate_adjusted_area(repj_raster, prj_districts, error)

In [37]:
def area_assessment_figure(input_f, output_f):
    '''
    takes in a csv of area assessment calculations
    and creates the columns and structure for the 
    publication table
    '''
    north = gpd.read_file('../../data/shapefiles/pd_north.shp')
    east = gpd.read_file('../../data/shapefiles/pd_east.shp')
    west = gpd.read_file('../../data/shapefiles/pd_west.shp')
    df = pd.read_csv(input_f)
    district_region = {}
    district_region['north'] = list(north.ADM2_EN)
    district_region['east'] = list(east.ADM2_EN)
    district_region['west'] = list(west.ADM2_EN)
    district_region = {district: region for region, districts in district_region.items() for district in districts}
    df['region'] = df['district'].map(district_region)
    df = df.sort_values(by='region')
    df_pubfigure2 = df[['region', 'district', 'Monoculture', 'Agroforestry', 'Natural']]
    df_pubfigure2.to_csv(output_f, index=False)
    return df_pubfigure2

In [38]:
area_assessment_figure('../../data/adj_area_assessment.csv',
                       '../../data/area_assessment_figure2.csv')

,region,district,Monoculture,Agroforestry,Natural
0,east,Adansi South,3327.0,41256.0,13115.0
22,east,Twifo Atti-Morkwa,13103.0,48430.0,20790.0
18,east,Sene West,204.0,41928.0,107211.0
17,east,Sekyere Afram Plains North,452.0,150808.0,75485.0
14,east,Kwahu West,484.0,25261.0,3603.0
13,east,Kwahu South,340.0,23900.0,12809.0
11,east,Kwahu Afram Plains South,209.0,56262.0,47434.0
10,east,Kwahu Afram Plains North,231.0,18639.0,14945.0
12,east,Kwahu East,363.0,23005.0,7567.0
1,east,Asante Akim South,2033.0,74822.0,11666.0


## Other // Not using

In [67]:
#calculate total area of raster (just for reference)
src = rs.open(repj_raster)
data = src.read(1)
valid_px = np.sum(data != src.nodata)
pixel_width = src.transform[0]  # X resolution
pixel_height = -src.transform[4]  # Y resolution
px_size = pixel_width * pixel_height
total_area_in_m2 = valid_px * (px_size)
total_area_in_ha = total_area_in_m2 / 10000

# sum all area assessments
# drops nodata column
district_total_area = df.iloc[:, :-2].sum().sum()

round(total_area_in_ha), round(district_total_area)

(5247876, 5247333)

In [70]:
df['ag_area'] = df.agroforestry + df.monoculture
df['nat_area'] = df.natural
df_pubfigure = df[['district', 'ag_area', 'nat_area']]
df_pubfigure

,district,ag_area,nat_area
0,Adansi South,54855.55,16628.07
1,Asante Akim South,98196.58,14791.46
2,Assin North,67500.35,2964.08
3,Atwima Mponua,164552.60,16005.69
4,Bawku West,16630.87,2381.25
5,Bosome Freho,42459.31,11938.37
6,Builsa South,25806.98,3711.35
7,Daffiama Bussie Issa,17875.03,19311.11
8,Juaben Municipal,12951.70,3137.66
9,Kasena Nankana West,20175.93,2528.68
